<a href="https://colab.research.google.com/github/HimashiBal/Predicting-Shark-Habitats/blob/main/env_after_mpm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **set up**

In [ ]:
!pip install netCDF4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import urllib.request
import xarray as xr
import netCDF4 as nc
import time
from matplotlib import pyplot as plt
from matplotlib.colors import LinearSegmentedColormap,BoundaryNorm,Normalize
#from mpl_toolkits.basemap import Basemap
from datetime import date,datetime

np.warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

df=pd.read_csv('/content/drive/MyDrive/Research_Project_Sharks/downloaded raw env data +raw mako shark/Shortfin Mako Vaudo et al - raw data.csv')
df.head(5)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,shark,timestamp,longitude,latitude,argos_location_class,Unnamed: 5
0,1,2013-03-23 12:00:00,-86.2900,21.2400,3,NaN
1,1,2013-03-23 19:23:36,-86.4973,21.4105,2,NaN
2,1,2013-03-23 19:32:53,-86.5063,21.4016,1,NaN
3,1,2013-03-23 21:04:34,-86.4888,21.4526,3,NaN
4,1,2013-03-24 12:00:30,-86.4793,22.0969,1,NaN


In [ ]:
df_mpm=pd.read_csv("/content/drive/MyDrive/Research_Project_Sharks/cleaned data/MPM_Data_correct.csv")

df_mpm.head()

,id,date,lon,lat,x,y,x.se,y.se,logit_g,logit_g.se,g
0,1,2013-03-23 12:00:00,-86.290063,21.239983,-9605.765859,2405.044227,0.215740,0.171328,1.668888,0.826140,0.841428
1,1,2013-03-24 12:00:00,-86.479020,22.096226,-9626.800504,2507.015854,0.798780,0.436279,1.716944,0.744235,0.847735
2,1,2013-03-25 12:00:00,-86.725278,22.679689,-9654.213813,2576.859322,17.613493,17.084095,1.733028,0.733996,0.849799
3,1,2013-03-26 12:00:00,-87.163946,23.257706,-9703.046034,2646.347950,26.413321,27.220071,1.716105,0.732660,0.847627
4,1,2013-03-27 12:00:00,-87.650941,23.762229,-9757.258107,2707.250445,24.040336,25.236523,1.632647,0.755455,0.836532


In [ ]:
df_mpm['YY_MM_DD'] = df_mpm['date'].str.slice(0,10)
YY_MM_DD = df_mpm.YY_MM_DD
df_mpm['HH_MM_SS'] = df_mpm['date'].str.slice(11,20)
HH_MM_SS = df_mpm.HH_MM_SS
date = df_mpm.date
lon=df_mpm.lon
lat=df_mpm.lat

# **Elevation redo**


In [ ]:
url_elev_2 = 'https://coastwatch.pfeg.noaa.gov/erddap/griddap/GEBCO_2020.csv?elevation%5B(12.98125):(42.789583)%5D%5B(-93.997917):(-64.189583)%5D&.draw=surface&.vars=longitude%7Clatitude%7Celevation&.colorBar=%7C%7C%7C%7C%7C&.bgColor=0xffccccff'
ulr_elev_base_2 = "https://coastwatch.pfeg.noaa.gov/erddap/griddap/GEBCO_2020.csv?elevation"

In [ ]:
x={}
dictionary = {'UTC':[''], 'm':[''], 'degrees_north':[''], 'degrees_east':[''],'m':['']  }
z = pd.DataFrame(dictionary)


# for i in range(1, len(date)):
for i in range(2):
  x[i] = lon[i]+180
  url_elev_new = ulr_elev_base_2+ "%5B("+str(lat[i])+"):1:("+str(lat[i])+")%5D%5B("+str(x[i])+"):1:("+str(x[i])+")%5D"
  # print(url_elev_new)
  print(i)
  new=pd.read_csv(url_elev_new,skiprows=1)


  frames = [z, new]
  z = pd.concat(frames)


print(z)


0
1
   UTC     m degrees_north degrees_east
0                                      
0  NaN  1913     21.239583    93.710417
0  NaN  1682     22.097917    93.522917


In [ ]:
z.to_csv('z_5000_8214.csv')


# **Sea Surface Temperature SST**


In [ ]:
base_url_sst = "https://coastwatch.pfeg.noaa.gov/erddap/griddap/erdHadISST.csv?sst"
dictionary = {'UTC':[''], 'degrees_north':[''], 'degrees_east':[''],'C':[''] }
sst = pd.DataFrame(dictionary)

# len(date)
for i in range(5):
    newer_url1 = base_url_sst + "%5B("+str(YY_MM_DD[i])+"T"+str(HH_MM_SS[i])+"Z)%5D%5B("+str(lat[i])+"):("+str(lat[i])+")%5D%5B("+str(lon[i])+"):("+str(lon[i])+")%5D&.draw=surface&.vars=longitude%7Clatitude%7Csst&.colorBar=%7C%7C%7C%7C%7C&.bgColor=0xffccccff"
    print(i)
    # print(newer_url1)
    d=pd.read_csv(newer_url1, skiprows=1)



    frames = [sst, d]
    sst = pd.concat(frames)

print(sst)


0
1
2
3
4
                    UTC degrees_north degrees_east          C
0                                                            
0  2013-03-16T12:00:00Z          21.5        -86.5  25.682531
0  2013-03-16T12:00:00Z          22.5        -86.5   25.39095
0  2013-03-16T12:00:00Z          22.5        -86.5   25.39095
0  2013-03-16T12:00:00Z          23.5        -87.5  24.639915
0  2013-03-16T12:00:00Z          23.5        -87.5  24.639915


In [ ]:
sst.head(5)

print(len(sst))

In [ ]:


sst.to_csv('trial')


In [ ]:
import time
import urllib.request
import urllib.error

def uptime_bot(url):
    while True:
        try:
            conn = urllib.request.urlopen(url)
        except urllib.error.HTTPError as e:
            # Email admin / log
            print(f'HTTPError: {e.code} for {url}')
        except urllib.error.URLError as e:
            # Email admin / log
            print(f'URLError: {e.code} for {url}')
        else:
            # Website is up
            print(f'{url} is up')
        time.sleep(60)

if __name__ == '__main__':
    url = 'http://www.google.com/py'
    uptime_bot(url)

# **Distance to Coast DTC**

In [ ]:
urlbase= "https://pae-paha.pacioos.hawaii.edu/erddap/griddap/dist2coast_1deg.csv?"
dictionary = {'degrees_north':[''], 'degrees_east':[''],'km':[''] }
dtc = pd.DataFrame(dictionary)

for i in range(8214,8220):
    urlnew = urlbase + "dist%5B("+str(lat[i])+"):("+str(lat[i])+")%5D%5B("+str(lon[i])+"):("+str(lon[i])+")%5D&.draw=surface&.vars=longitude%7Clatitude%7Cdist&.colorBar=%7C%7C%7C%7C%7C&.bgColor=0xffccccff"
    print(i)
    d=pd.read_csv(urlnew, skiprows=1)



    frames = [dtc, d]
    dtc = pd.concat(frames)

print(dtc)#distance to coast

KeyError: ignored

In [ ]:
urlbase= "https://pae-paha.pacioos.hawaii.edu/erddap/griddap/dist2coast_1deg.csv?"
dictionary = {'degrees_north':[''], 'degrees_east':[''],'km':[''] }
dtc = pd.DataFrame(dictionary)

for i in range(100):
    urlnew = urlbase + "dist%5B("+str(lat[i])+"):("+str(lat[i])+")%5D%5B("+str(lon[i])+"):("+str(lon[i])+")%5D&.draw=surface&.vars=longitude%7Clatitude%7Cdist&.colorBar=%7C%7C%7C%7C%7C&.bgColor=0xffccccff"
    print(i)
    d=pd.read_csv(urlnew, skiprows=1)



    frames = [dtc, d]
    dtc = pd.concat(frames)

print(dtc)#distance to coast


In [ ]:
dtc.head(5)

,degrees_north,degrees_east,km
0,,,
0,21.24,-86.29,44
0,22.1,-86.48,71
0,22.68,-86.73,124
0,23.26,-87.16,182


In [ ]:
dtc.to_csv('dtc_3691_8214.csv', index=False)

In [ ]:
print(dtc["km"])

0      
0    24
0    22
0    38
0    44
     ..
0    12
0    13
0    14
0    18
0    24
Name: km, Length: 2215, dtype: object


# **Elevation**

In [ ]:
url_elev = "https://coastwatch.pfeg.noaa.gov/erddap/griddap/srtm30plus.csv?z%5B(-75.8125):(26.004166666666663)%5D%5B(207.47916666666666):(252.00416666666666)%5D&.draw=surface&.vars=longitude%7Clatitude%7Cz&.colorBar=%7C%7C%7C%7C%7C&.bgColor=0xffccccff"
ulr_elev_base = "https://coastwatch.pfeg.noaa.gov/erddap/griddap/srtm30plus.csv?"

In [ ]:
x={}
dictionary = {'UTC':[''], 'm':[''], 'degrees_north':[''], 'degrees_east':[''],'m':['']  }
z = pd.DataFrame(dictionary)


# for i in range(1, len(date)):
for i in range(5000,8500):
  x[i] = lon[i]+180
  url_elev_new = ulr_elev_base+ "z%5B("+str(lat[i])+"):1:("+str(lat[i])+")%5D%5B("+str(x[i])+"):1:("+str(x[i])+")%5D"
  # print(url_elev_new)
  print(i)
  new=pd.read_csv(url_elev_new,skiprows=1)


  frames = [z, new]
  z = pd.concat(frames)


print(z)


In [ ]:
z.to_csv('z_5000_8214.csv')


# **Salinity**

In [ ]:
import time

In [ ]:
salinty_url ='https://coastwatch.pfeg.noaa.gov/erddap/griddap/coastwatchSMOSv662SSS3day.csv?sss'
dictionary = {'UTC':[''], 'm':[''], 'degrees_north':[''], 'degrees_east':[''],'PSU':['']  }
sss = pd.DataFrame(dictionary)



for i in range(2486):
    url=salinty_url+"%5B("+str(YY_MM_DD[i])+"T"+str(HH_MM_SS[i])+"Z):1:("+str(YY_MM_DD[i])+"T"+str(HH_MM_SS[i])+"Z)%5D%5B(0.0)%5D%5B("+str(lat[i])+"):1:("+str(lat[i])+")%5D%5B("+str(lon[i])+"):1:("+str(lon[i])+")%5D"
    print(url)
    print(i)
    time.sleep(0.01)
    new=pd.read_csv(url,skiprows=1)



    frames = [sss, new]
    sss = pd.concat(frames)
print(sss)

# sss.to_csv('sss_0_3000.csv')

In [ ]:
sss.to_csv('sss_0_2486_redo.csv')

In [ ]:
print(len(sss))

NameError: ignored

In [ ]:
sss.to_csv('sss_0_2000.csv')

# **Chlorophyll**


In [ ]:
chld = "https://coastwatch.pfeg.noaa.gov/erddap/griddap/erdMH1chlamday.csv?chlorophyll"
dictionary = {'UTC':[''], 'degrees_north':[''], 'degrees_east':[''],'mg m-3':['']  }
chl = pd.DataFrame(dictionary)



# # for i in range(1, len(timestamp)):
for i in range(4000,7000):
    url=chld+"%5B("+str(YY_MM_DD[i])+"):1:("+str(YY_MM_DD[i])+")%5D%5B("+str(lat[i])+"):1:("+str(lat[i])+")%5D%5B("+str(lon[i])+"):1:("+str(lon[i])+")%5D"
    # print(url)
    print(i)
    new=pd.read_csv(url,skiprows=1)



    frames = [chl, new]
    chl = pd.concat(frames)


print(chl)



In [ ]:
chl.to_csv('chl_4000_7000.csv')

In [ ]:
# chl.rename(columns = {'mg m^-3':'chlr'}, inplace = True)


CHLR = chl.iloc[:,-1]
print(CHLR)

SST = sst.iloc[:,-1]
print(SST)

DTC = dtc.iloc[:,-1]
print(DTC)

SSS = sss.iloc[:,-1]
print(SSS)

Z = z.iloc[:,-1]
print(Z)

In [ ]:
# import math
# for i in range(len(date)):#change what we take length of for greatest number
# #import the combined mpm env dataset
#    if chl[i]== "" or chl[i]== "nul": # search for all possible blanks


#        #d_f = math.sqrt((chl))

In [ ]:
DTC = dtc.iloc[:,-1]
print(DTC)

